In [33]:
import os, json, pickle, csv, pandas
import numpy as np
from classDeclarations import file_data
from random import sample
MAX_SAMPLE_SIZE = 100

In [34]:
file_extensions = []
with open("../file_extensions.txt", "r") as fp_ext:
    file_extensions = [line.rstrip() for line in fp_ext]


file_extensions = [''] + [x.lower() for x in file_extensions]
print(file_extensions)

['', '0', '23', 'csv', 'doc', 'ds_store', 'otl', 'pdf', 'txt', 'temporaryitems', 'xls', 'zip', '_hydrochem_data', '_pacificasource', '_readme', '_crossover_plots', 'apdisk', 'avi', 'bmp', 'cdf', 'cfg', 'cfl', 'cid', 'csv', 'csv~', 'dat', 'db', 'dbf', 'des', 'dob', 'doc', 'docx', 'exe', 'fig', 'final', 'for', 'geos', 'gif', 'gmtcommands4', 'gz', 'hob', 'htm', 'html', 'idv', 'info', 'inp', 'inv', 'jnl', 'jpg', 'kmz', 'log', 'lst', 'm', 'mat', 'mp4', 'm~', 'nas', 'nc', 'note', 'pdf', 'png', 'ppt', 'prj', 'prn', 'ps', 'py', 'qc', 'rtf', 'save', 'sec', 'shp', 'shx', 'sta', 'tas', 'tsv', 'txt', 'txt~', 'uni', 'var', 'xls', 'xlsx', 'xml']


In [35]:
Path = "../../CDIACPub8"

imgs = []
tabular = []
freetext = []
json_xml = []
netcdf = []
unknown = []

missing_count = 0
df = pandas.read_csv('../CDIACFileData/labels/cdiac_naivetruth_processed.csv')


labelSeriesObj = df['file_label']
pathSeriesObj = df['path']


for i in range(labelSeriesObj.size):
    if labelSeriesObj[i] == 'image':
        imgs.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'tabular':
        tabular.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'freetext':
        freetext.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'json/xml':
        json_xml.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'netcdf':
        netcdf.append(pathSeriesObj[i])
    else: 
        unknown.append(pathSeriesObj[i])


In [54]:
print("loading files now...")

with open('../CDIACFileData/one_gram_distr/one_grams_distr_sparse_256KB.pkl', "rb") as fp1:
    one_gram = pickle.load(fp1)
with open('../CDIACFileData/two_gram_distr/two_grams_distr_sparse_256KB.pkl', "rb") as fp2:
    two_gram = pickle.load(fp2)

print("loading files done!")
#byte_distr = pandas.DataFrame.from_dict(one_gram)
#two_grams_dicts = pandas.DataFrame.from_dict(two_gram)
#best_extractors = json.load(open("../CorrelatingExtractors/best_extractors.json"))


loading files now...
loading files done!


In [55]:
one_gram_new_index = one_gram.index.tolist()
one_gram_new_index.append("file_type")
one_gram_new_index.append("file_size")
one_gram = one_gram.reindex(one_gram_new_index)

two_gram_dicts_new_index = two_gram.index.tolist()
two_gram_dicts_new_index.append("file_type")
two_gram_dicts_new_index.append("file_size")
two_gram = two_gram.reindex(two_gram_dicts_new_index)



In [56]:
print('Doing one gram')
one_gram = one_gram.sparse.to_dense()
print('moving onto two_gram...')
two_gram = two_gram.sparse.to_dense()
print('Done!')

Doing one gram
moving onto two_gram...
Done!


In [60]:
img_sample_names = sample(imgs, MAX_SAMPLE_SIZE)
tabular_sample_names = sample(tabular, MAX_SAMPLE_SIZE)
freetext_sample_names = sample(freetext, MAX_SAMPLE_SIZE)
json_xml_sample_names = sample(json_xml, MAX_SAMPLE_SIZE)
netcdf_sample_names = sample(netcdf, MAX_SAMPLE_SIZE)
unknown_sample_names = sample(unknown, MAX_SAMPLE_SIZE)

In [61]:
img_data = []
tabular_data = []
freetext_data = []
json_xml_data = []
netcdf_data = []
unknown_data = []
dataset = [img_data, tabular_data, freetext_data, json_xml_data, netcdf_data, unknown_data]

for idx, data_list in enumerate(dataset):
    curr_sample = []
    if idx == 0:
        best_extractor = 'image'
        curr_sample = img_sample_names
    elif idx == 1:
        best_extractor = 'tabular'
        curr_sample = tabular_sample_names
    elif idx == 2:
        best_extractor = 'freetext'
        curr_sample = freetext_sample_names
    elif idx == 3:
        best_extractor = 'json/xml'
        curr_sample = json_xml_sample_names
    elif idx == 4:
        best_extractor = 'netcdf'
        curr_sample = netcdf_sample_names
    elif idx == 5:
        best_extractor = 'unknown'
        curr_sample = unknown_sample_names
    print("Done with index", idx)

    for file_name in curr_sample:
        #best_extractor_row_index = df.path[df.path==file_name].index.tolist()

        name, extension = os.path.splitext(file_name) # file_name actually holds file path
        extension = extension[1:].lower()

        one_gram.at['file_type', file_name] = file_extensions.index(extension)
        two_gram.at['file_type', file_name] = file_extensions.index(extension)
          
        one_gram.at['file_size', file_name] = os.path.getsize(file_name)
        two_gram.at['file_size', file_name] = os.path.getsize(file_name)

        curr_file_data = file_data(file_name, one_gram[file_name], two_gram[file_name], best_extractor)
        data_list.append(curr_file_data)

Done with index 0
Done with index 1
Done with index 2
Done with index 3
Done with index 4
Done with index 5


In [62]:
print('Dumping!')

with open('gathered_data_distr/sample3/byte_distr_sample_256KB_v3', 'wb+') as handle:
    pickle.dump(dataset, file=handle, protocol=pickle.HIGHEST_PROTOCOL)
print('Dumped.')



Dumping!
Dumped.
